<a href="https://colab.research.google.com/github/SaiRekha555/BharatIntern-Task1/blob/main/News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-telegram-bot requests


In [ ]:
import logging
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import requests

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Replace 'YOUR_TELEGRAM_BOT_TOKEN' with your bot token
TELEGRAM_BOT_TOKEN = '7333499420:AAGT3nIkr6Tt2EXD0aPSVVnHc2tBJYJBico'
# Replace 'YOUR_NEWSAPI_KEY' with your NewsAPI key
NEWSAPI_KEY = 'pub_459491e2876a221f90221242c6c947efa2b40'

# Function to fetch news from NewsAPI
def fetch_news(category=None, country='us'):
    url = f'https://newsapi.org/v2/top-headlines?apiKey={NEWSAPI_KEY}&country={country}'
    if category:
        url += f'&category={category}'
    response = requests.get(url)
    articles = response.json().get('articles', [])
    return articles

# Function to format the news articles
def format_news(articles):
    if not articles:
        return "No news available at the moment."
    formatted_news = []
    for article in articles[:5]:  # Limiting to top 5 articles
        title = article.get('title')
        description = article.get('description')
        url = article.get('url')
        formatted_news.append(f"📰 *{title}*\n{description}\n[Read more]({url})\n")
    return "\n\n".join(formatted_news)

# Command handlers
def start(update: Update, context: CallbackContext) -> None:
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\! I am your news bot\. Use /news to get the latest news\.',
        reply_markup=ForceReply(selective=True),
    )

def help_command(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Use /news [category] to get the latest news. Categories include general, business, entertainment, health, science, sports, technology.')

def news(update: Update, context: CallbackContext) -> None:
    category = ' '.join(context.args).lower()
    valid_categories = ['general', 'business', 'entertainment', 'health', 'science', 'sports', 'technology']
    if category not in valid_categories:
        category = None
    articles = fetch_news(category)
    formatted_news = format_news(articles)
    update.message.reply_text(formatted_news, parse_mode='Markdown')

def main() -> None:
    updater = Updater(TELEGRAM_BOT_TOKEN)
    dispatcher = updater.dispatcher

    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))
    dispatcher.add_handler(CommandHandler("news", news))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()
